# Sélection des variables - Stepwise

## 1. Principe

Lorsque le dataset comporte $p$ covariables, il ne serait pas judicieux de trouver les $\binom{p}{k}$ covariables qui rendent le modèle optimal à la main. Pour cela, on utilise une méthode de type **stepwise**. C'est un type de méthode itératives de sélection des covariables qui ajoute ou retire les covariables en fonction des critères de performances du modèles. Les critères utilisés sont :

1. AIC
2. BIC
3. $p$-Value (test de Wald)

Le stepwise comporte trois variantes :

1. Forward selection
2. Backward elimination
3. Stepwise mixte

### 1.1 Forward selection
Comme son nom l'indique, on part d'un modèle avec 0 covariables et on les ajoute une à une en cherchant à trouver le sous-jeu de covrariables qui ont les meilleurs résultats en terme de métriques (critères de performance). On cherche à estimer le modèle $\mathcal{M}_{k}$ de la forme :

$$
\log{(\frac{P(Y = 1 | X)}{1 - P(Y = 1 | X)})} = \beta_{0} + \sum_{i=1}^{k}\beta_{i}X_{i}
$$

de telle sorte qu'on cherche le sous-ensemble $(X_{i})_{i \in ⟦1; k⟧} \in (X_{i})_{i \in ⟦1; p⟧}, k ≤ p$ qui a les meilleurs critères de performance en ajoutant les $X_{j}, j \in ⟦k + 1; p⟧$ un à un. Remarque : $\mathcal{M}_{k}$ est une régression logistique de degré $k$ sur un dataset de $p$ variables. Le modèle a la relation de récurrence suivante :

$$
\forall k ≤ p \in \mathbb{N}, \textcolor{white}{.} \mathcal{M}_{k+1} = \mathcal{M}_{k} + X_{j}
$$
Le modèle initial $\mathcal{M}_{0}$ est de la forme : $\mathcal{M}_{0} : \log{(\frac{p}{1 - p})} = \beta_{0}$ avec $P(Y = 1 | X) = p$. A chaque itération, le modèle prend une variable $X_{j}$ **pas encore sélectionnée** et on définit **le modèle avec la variable $X_{j}$ ajoutée** $\mathcal{M}_{k,j}$

$$
\forall k ≤ p \in \mathbb{N}, \textcolor{white}{.} \mathcal{M}_{k,j} = \mathcal{M}_{k} + X_{j}
$$

puis on détermine les métriques de performance associées au modèle :

$$
AIC = -2\mathcal{l}(\beta) +2k
$$
$$
BIC = -2\mathcal{l}(\beta) +2\log{(n)}
$$
$$
p-value
$$

Ou $\mathcal{l}(\beta)$ est la log-vraisemblance (log-likelihood).

Si le modèle avec la variable $X_{j}$ ajoutée, $\mathcal{M}_{k,j}$, a de meilleures performances que le modèle à l'instant $k$, $\mathcal{M}_{k}$, alors le modèle s'incrémente :

$$
\mathcal{M}_{k+1} = \mathcal{M}_{k} + X_{j}
$$

Sinon, l'algorithme s'arrête : on a trouvé le sous-ensemble $(X_{i})_{i \in ⟦1; k⟧} \in (X_{i})_{i \in ⟦1; p⟧}, k ≤ p$ optimal.

### 1.2 Backward elimination



### 1.3 Stepwise mixte

## 2. Un exemple en Python
---
### 2.1 Import des librairies nécéssaires

In [28]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import sys

from sklearn.metrics import mean_absolute_percentage_error, root_mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

from sklearn.linear_model import LinearRegression

from statsmodels.api import OLS
from statsmodels.tools import add_constant

Fonctions personnalisées

In [32]:
directory = '/home/alexis/python/.venv/PipelineDataScience/TimeSeries/Models/src/python'
sys.path.append(directory)

import _regression_lineaire as lr

### 2.2 Import / initialisation des données
#### 2.2.1 Import
On prend ici le dataset ozone_complet.csv.

In [33]:
data = pd.read_csv(directory + 'ozone_complet.csv',
                   header    = 0,
                   index_col = 0,
                   sep       = ';',
                   decimal   = ',')

data.info()

FileNotFoundError: [Errno 2] No such file or directory: '/home/alexis/python/.venv/PipelineDataScience/TimeSeries/Models/src/pythonozone_complet.csv'

In [30]:
data.head()

,maxO3,T6,T9,T12,T15,T18,Ne6,Ne9,Ne12,Ne15,...,Vdir9,Vvit9,Vdir12,Vvit12,Vdir15,Vvit15,Vdir18,Vvit18,Vx,maxO3v
date,,,,,,,,,,,,,,,,,,,,,
19950401,47.6,10.1,11.6,13.3,13.6,12.2,8.0,8.0,8.0,8.0,...,290.0,4.0,300.0,4.0,340.0,4.0,20.0,4.0,-3.4641,62.2
19950402,56.2,9.5,9.4,13.8,17.4,16.3,8.0,8.0,7.0,0.0,...,160.0,2.0,180.0,3.0,110.0,1.0,350.0,2.0,0.0000,47.6
19950403,61.8,3.6,8.0,16.8,21.5,20.2,4.0,5.0,2.0,2.0,...,20.0,2.0,340.0,1.0,170.0,2.0,170.0,3.0,-0.3420,56.2
19950404,50.8,9.5,10.5,11.4,12.2,11.4,8.0,7.0,7.0,7.0,...,10.0,4.0,350.0,3.0,350.0,3.0,350.0,4.0,-0.5209,61.8
19950405,59.8,9.8,10.8,13.8,14.3,13.3,8.0,7.0,8.0,8.0,...,340.0,2.0,280.0,1.0,320.0,3.0,350.0,4.0,-0.9848,50.8


#### 2.2.2 Gestion des données manquantes

In [31]:
missing_percentage = data.isna().mean() * 100

print("MISSING VALUES :")
if missing_percentage[missing_percentage != 0].empty:
    print("No")
else:
    print(missing_percentage[missing_percentage != 0].sort_values(ascending=False))

data.dropna(inplace=True)

MISSING VALUES :
maxO3     4.986339
maxO3v    4.986339
Ne18      0.341530
Ne9       0.136612
Ne12      0.136612
Ne6       0.136612
Ne15      0.136612
T18       0.068306
T15       0.068306
T12       0.068306
T9        0.068306
T6        0.068306
Vdir6     0.068306
Vdir9     0.068306
Vvit6     0.068306
Vvit9     0.068306
Vdir12    0.068306
Vdir15    0.068306
Vvit12    0.068306
Vvit15    0.068306
Vdir18    0.068306
Vvit18    0.068306
Vx        0.068306
dtype: float64


#### 2.2.3 Gestion des variables 

In [32]:
target = 'maxO3'

y = data[target]
X = data.drop(target, axis=1)

#### 2.2.4 Séparation des données test / train

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

### 2.3 Entraînement des modèles
#### 2.3.1 Forward selection
La commande verbose=True permet de voir ce qui se passe lors que l'appel de la méthode

In [34]:
linreg_forward = lr.linreg_forward_proc(
    add_constant(X_train),
    y_train,
    crit='BIC',
    verbose=True
)

Selected predictors: ['maxO3v', 'const'] AIC: 9293.166181212824 BIC: 9303.157713525434
Selected predictors: ['maxO3v', 'Ne12', 'const'] AIC: 8993.331519924703 BIC: 9008.318818393616
Selected predictors: ['maxO3v', 'Ne12', 'T15', 'const'] AIC: 8938.813102067257 BIC: 8958.796166692477
Selected predictors: ['maxO3v', 'Ne12', 'T15', 'T6', 'const'] AIC: 8862.179225265294 BIC: 8887.158056046817
Selected predictors: ['maxO3v', 'Ne12', 'T15', 'T6', 'T12', 'const'] AIC: 8842.648940532536 BIC: 8872.623537470365
Selected predictors: ['maxO3v', 'Ne12', 'T15', 'T6', 'T12', 'Vdir12', 'const'] AIC: 8824.488089130267 BIC: 8859.4584522244
Selected predictors: ['maxO3v', 'Ne12', 'T15', 'T6', 'T12', 'Vdir12', 'Vvit6', 'const'] AIC: 8813.623360883834 BIC: 8853.589490134273
Selected predictors: ['maxO3v', 'Ne12', 'T15', 'T6', 'T12', 'Vdir12', 'Vvit6', 'Vdir9', 'const'] AIC: 8809.145216000523 BIC: 8854.107111407266


#### 2.3.2 Backward elimination
C'est l'inverse du forward selection : on part des $p$ variables $X_{i}$ et on les retire une à une jusqu'à trouver le sous-jeu de covariables qui ont les meilleurs résultats en terme de métriques (critères de performance).

In [35]:
linreg_backward = lr.linreg_backward_proc(
    add_constant(X_train),
    y_train,
    crit='BIC',
    verbose=True
)

Selected predictors: ['Ne12', 'Ne15', 'Ne18', 'Ne6', 'Ne9', 'T12', 'T15', 'T18', 'T6', 'T9', 'Vdir12', 'Vdir15', 'Vdir18', 'Vdir6', 'Vdir9', 'Vvit15', 'Vvit18', 'Vvit6', 'Vvit9', 'Vx', 'maxO3v', 'const'] AIC: 8808.589067372672 BIC: 8918.495922811378
Selected predictors: ['Ne12', 'Ne18', 'Ne6', 'Ne9', 'T12', 'T15', 'T18', 'T6', 'T9', 'Vdir12', 'Vdir15', 'Vdir18', 'Vdir6', 'Vdir9', 'Vvit15', 'Vvit18', 'Vvit6', 'Vvit9', 'Vx', 'maxO3v', 'const'] AIC: 8806.605109066184 BIC: 8911.516198348585
Selected predictors: ['Ne12', 'Ne18', 'Ne6', 'Ne9', 'T12', 'T15', 'T18', 'T6', 'Vdir12', 'Vdir15', 'Vdir18', 'Vdir6', 'Vdir9', 'Vvit15', 'Vvit18', 'Vvit6', 'Vvit9', 'Vx', 'maxO3v', 'const'] AIC: 8804.812783823741 BIC: 8904.728106949839
Selected predictors: ['Ne12', 'Ne18', 'Ne9', 'T12', 'T15', 'T18', 'T6', 'Vdir12', 'Vdir15', 'Vdir18', 'Vdir6', 'Vdir9', 'Vvit15', 'Vvit18', 'Vvit6', 'Vvit9', 'Vx', 'maxO3v', 'const'] AIC: 8802.936358566345 BIC: 8897.855915536136
Selected predictors: ['Ne12', 'Ne18', 'Ne9'

#### 2.3.2 Stepwise mixte
C'est une combinaison linéaire des deux méthodes

In [36]:
linreg_stepwise = lr.linreg_stepwise_proc(
    add_constant(X_train),
    Y_train,
    crit='BIC',
    verbose=True
)

ValueError: endog and exog matrices are different sizes

### 2.4 Covariables que les modèles ont sélectionnés

In [1]:
linreg_forward.model.exog_names

NameError: name 'linreg_forward' is not defined

In [ ]:
linreg_backward.model.exog_names

### 2.5 Prédictions

In [ ]:
y_test_pred_linreg_forward = linreg_forward.predict(add_constant(X_test)[linreg_forward.model.exog_names]) # Forward selection
y_test_pred_linreg_backward = linreg_backward.predict(add_constant(X_test)[linreg_backward.model.exog_names]) # Backward selection

### 2.6 Evalusation des performances du modèle

**Forward selection**

In [ ]:
linreg_forward.bic

In [ ]:
linreg_forward.aic

In [ ]:
linreg_forward.bse

In [ ]:
root_mean_squared_error(y_test, y_test_pred_linreg_forward)

**Backward elimination**

In [ ]:
linreg_backward.bic

In [ ]:
linreg_backward.aic

In [ ]:
linreg_backward.bse

In [ ]:
root_mean_squared_error(y_test, y_test_pred_linreg_backward)

**Stepwise mixte**